| **Author**          | **Roll No**   | **Version** |
|---------------------|----------------|--------------|
| Kaloori Shiva Prasad        | 24250041     | 1.0          |


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time  
from requests.exceptions import ChunkedEncodingError, ConnectionError


base_url = 'https://www.gutenberg.org'

def get_book_urls_from_author(letter):
    response = requests.get(f'{base_url}/browse/authors/{letter}')
    soup = BeautifulSoup(response.text, 'html.parser')
    book_urls = []
    for li in soup.find_all('li', class_='pgdbetext'):
        a_tag = li.find('a', href=True)
        if a_tag:
            book_url = base_url + a_tag['href']
            if '(English)' in li.text: 
                book_urls.append(book_url)
    print(f'{letter} : {len(book_urls)}')
    return book_urls

def get_book_id(book_url):
   
    return book_url.strip('/').split('/')[-1]

def download_book(book_url, save_dir='mnt/HDFS1/language_nlp/english_nlp_team17/Assignment1/Data_Shiva/Project Gutenberg Data'):
    os.makedirs(save_dir, exist_ok=True)  
    base_text_url = 'https://www.gutenberg.org/cache/epub'
    book_id = get_book_id(book_url)
    download_url = f'{base_text_url}/{book_id}/pg{book_id}.txt'
    
    try:
        book_response = requests.get(download_url, stream=True) 
        if book_response.status_code == 200:  
            
            file_path = os.path.join(save_dir, f'{book_id}.txt')
            with open(file_path, 'wb') as file:
                for chunk in book_response.iter_content(chunk_size=1024):
                    if chunk:  
                        file.write(chunk)
            # print(f"Successfully downloaded {book_id}.txt")
        else:
            print(f"Failed to download from {download_url}. Status code: {book_response.status_code}")
    
    except (ChunkedEncodingError, ConnectionError) as e:
        print(f"Error occurred while downloading {download_url}: {e}. Skipping to next.")
    except Exception as e:
        print(f"Unexpected error occurred: {e}. Skipping to next.")


# def download_book(book_url, save_dir='Project Gutenberg Data'):
#     os.makedirs(save_dir, exist_ok=True)  # Create the directory if it does not exist
#     base_text_url = 'https://www.gutenberg.org/cache/epub'
#     book_id = get_book_id(book_url)
#     download_url = f'{base_text_url}/{book_id}/pg{book_id}.txt'
#     book_response = requests.get(download_url)
#     if book_response.status_code == 200:  # Check if the download is successful
#         # Save the book text to a file
#         file_path = os.path.join(save_dir, f'{book_id}.txt')
#         with open(file_path, 'wb') as file:
#             file.write(book_response.content)
#     else:
#         print(f"Failed to download from {download_url}. Status code: {book_response.status_code}")
# Main function to scrape books
def scrape_books_from_authors(start_letter, end_letter):
    for letter in range(ord(start_letter), ord(end_letter) + 1):
        book_urls = get_book_urls_from_author(chr(letter))
        book_urls_b = book_urls
        k = 0
        for book_url in book_urls_b:
            download_book(book_url)
            k = k + 1
            if k%10 == 0:
                print(f'File downloded from : {book_url}, k = {k}')
            time.sleep(2)  
if __name__ == "__main__":
    scrape_books_from_authors('s', 'z')  
